In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os
from src.investing import Investing
import pandas as pd
import numpy as np

In [3]:
ivt = Investing()

ivt.getFinancialAll()

7217 done
7218 done
7219 done
7220 done
7222 done
7224 done
7226 done
7228 done
7229 done
7231 done
7235 done
7236 done
7238 done
7239 done
7240 done
7241 done
7242 done
7244 done
7245 done
7246 done
7247 done
7250 done
7254 done
7255 done
7256 done
7259 done
7261 done
7264 done
7265 done


In [ ]:
ivt = Investing()

ivt.getFinancialAll()